# LangGraph Persistence and Human-in-the-Loop (HITL)

## 1. Why Persistence?
By default, an agent's state is lost when the script ends. Persistence allows you to:
- **Save Progress**: Resume a multi-step task later.
- **Thread Management**: Have separate "remembered" conversations with multiple users.
- **Fault Tolerance**: Recover if the system crashes mid-task.

## 2. Setup
We use `langgraph` and a persistent checkpointer (like `SqliteSaver`).

## 3. Defining a Persistent Graph
We will use `SqliteSaver` to store checkpoints in a local file.

In [ ]:
%pip install -qU langgraph langchain-openai

## 4. Defining a Persistent Graph
We will use `SqliteSaver` to store checkpoints in a local file.

In [ ]:
import sqlite3
from typing import Annotated
from typing_extensions import TypedDict
from langgraph.graph import StateGraph, START, END
from langgraph.graph.message import add_messages
from langgraph.checkpoint.sqlite import SqliteSaver

class State(TypedDict):
    messages: Annotated[list, add_messages]
    approved: bool

def chatbot(state: State):
    return {"messages": [("ai", "I have prepared a draft. Do you approve?")]}

def action_node(state: State):
    print("--- EXECUTION ACTION ---")
    return {"messages": [("ai", "Action executed successfully!")]}

builder = StateGraph(State)
builder.add_node("chatbot", chatbot)
builder.add_node("action", action_node)

builder.add_edge(START, "chatbot")

builder.add_edge("chatbot", "action")
builder.add_edge("action", END)

# Set up the persistent memory
conn = sqlite3.connect("checkpoints.db", check_same_thread=False)
memory = SqliteSaver(conn)

# Compile the graph with a checkpointer and an interrupt
graph = builder.compile(
    checkpointer=memory,
    interrupt_before=["action"]
)

## 5. Running with Threads
We use a `thread_id` to track the conversation.

In [ ]:
config = {"configurable": {"thread_id": "user_123"}}

# Start the graph
for event in graph.stream({"messages": [("user", "Hi!")]}, config):
    for node, values in event.items():
        print(f"Node: {node}")
        for msg in values.get("messages", []):
            msg.pretty_print()

## 6. Resuming After Interrupt
The graph paused before `action`. We can see the current state, and then continue.

In [ ]:
snapshot = graph.get_state(config)
print(f"Next node to run: {snapshot.next}")

# To resume, we call stream with None as input
for event in graph.stream(None, config):
    for node, values in event.items():
        print(f"Node: {node}")
        for msg in values.get("messages", []):
            msg.pretty_print()

## Summary
1.  **Persistence**: Use `SqliteSaver` to keep state alive.
2.  **Threads**: Use `thread_id` to separate different users.
3.  **HITL**: Use `interrupt_before` or `interrupt_after` to force human approval.
4.  **Resumption**: Continue a paused graph by passing `None` to `.stream()` or `.invoke()`.